In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from IPython import display
display.clear_output()


import supervision as sv
print("supervision.__version__:", sv.__version__)

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('best.pt')

In [ ]:
# # dict maping class_id to class_name
# CLASS_NAMES_DICT = model.model.names
# print(CLASS_NAMES_DICT)
# # class_ids of interest - person
# selected_classes = [3]

In [ ]:
import numpy as np

In [ ]:
# # create frame generator
# generator = sv.get_video_frames_generator('vid1.mp4')
# # create instance of BoxAnnotator
# box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
# # acquire first video frame
# iterator = iter(generator)
# frame = next(iterator)
# # model prediction on single frame and conversion to supervision Detections
# results = model(frame, verbose=False, conf=0.7)[0]

# # convert to Detections
# detections = sv.Detections.from_ultralytics(results)
# # only consider class id from selected_classes define above
# detections = detections[np.isin(detections.class_id, selected_classes)]

# # format custom labels
# labels = [
#     f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
#     for _, _, confidence, class_id, _ in detections
# ]

# # annotate and display frame
# anotated_frame=box_annotator.annotate(scene=frame, detections=detections, labels=labels)

# %matplotlib inline
# sv.plot_image(anotated_frame, (16,16))

In [ ]:
from ultralytics import YOLO
from custom_object_counter import * 
import cv2

model = YOLO('best.pt')
cap = cv2.VideoCapture('/home/mind/projects/projects/object_counting_misc/videos/vid1.avi')
# assert cap.isOpened(), "Error reading video file"

video_writer = cv2.VideoWriter("/home/mind/projects/projects/object_counting_misc/output_videos/output-vid1.avi",                               
                               cv2.VideoWriter_fourcc(*'mp4v'),
                               int(cap.get(5)),
                               (int(cap.get(3)), int(cap.get(4))))

classes_to_count = [0, 1, 2, 3, 4]
counter = ObjectCounter()  # Init Object Counter
# region_points = [(20, 400), (1080, 404), (1080, 360), (20, 360)]
region_points = [(0, 0), (0, 0), (0, 0), (0, 0)]
counter.set_args(view_img=True,
                 reg_pts=region_points,
                 classes_names=model.model.names,
                 draw_tracks=False)

while cap.isOpened():
  success, im0 = cap.read()
  if not success:
    print("Video frame is empty or video processing has been successfully completed.")
    break
  tracks = model.track(im0, persist=True, show=False,
                        classes=classes_to_count, conf=0.7, verbose=False)
  # print(tracks)
  im0 = counter.start_counting(im0, tracks)
  video_writer.write(im0)

video_writer.release()
cv2.destroyAllWindows()